In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "5765a06e",
   "metadata": {},
   "source": [
    "# Spark Aufgaben\n",
    "1. Importe laden\n",
    "2. Jupyter Spark starten und Twitter-Streams von Avro lesen\n",
    "3. ETL Strecke: Avro Daten einlesen und als Delta Datei wieder raus schreiben\n",
    "4. Analyse-Aufgaben erledigen \n",
    "5. Verlaufsanalyse durchführen\n",
    "6. **Ausschalten der Spark-App**\n",
    "\n",
    "## Wichtige Hinweise\n",
    "1. Führe alle Anweisungen in der vorgegebenen Reihenfolge aus. Die einzelnen Programmierzellen bauen aufeinander auf.\n",
    "2. **Beende unbedingt am Ende die Spark-Anwendung mit dem untersten Befehl \"spark.stop()\" , wenn du aufhörst an den Daten zu arbeiten.**\n",
    "3. Du kannst jederzeit das Notebook wieder hochfahren, wenn du Schritt 1 & 2 (Laden der Imports & Jupyter Spark und seine Konfigurationen hochfahren) ausführen.\n",
    "4. Mit **\"Strg\" + \"Enter\"** führst du einzelne Zellen direkt aus.\n",
    "5. In der oberen Leiste kannst du über **\"Insert\"** weitere Zellen hinzufügen, um weitere Test-Funktionen zu schreiben. "
   ]
  },
  {
   "cell_type": "markdown",
   "id": "03c9d1f3",
   "metadata": {},
   "source": [
    "## 1. Laden der Imports\n",
    "Hier werden alle benötigten Libraries für dieses Lab heruntergeladen."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "eebe3ff2",
   "metadata": {},
   "outputs": [],
   "source": [
    "from pyspark import SparkContext, SparkConf\n",
    "from pyspark.sql import SparkSession\n",
    "from pyspark.sql import SQLContext\n",
    "from pyspark.sql.types import *\n",
    "from pyspark.sql.window import Window\n",
    "from pyspark.sql import Row\n",
    "from pyspark.sql.functions import explode\n",
    "from pyspark.sql.functions import lower, col\n",
    "import pyspark.sql.functions as f\n",
    "\n",
    "from delta import *\n",
    "\n",
    "\n",
    "import datetime\n",
    "from datetime import datetime\n",
    "import json\n",
    "\n",
    "\n",
    "# use 95% of the screen for jupyter cell\n",
    "from IPython.core.display import display, HTML\n",
    "display(HTML(\"<style>.container {width:100% !important; }<style>\"))"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "2579b387",
   "metadata": {},
   "source": [
    "## 2. Jupyter Spark & Konfigurationen hochfahren\n",
    "Hier wird die App jupyter-spark konfiguriert und hochgefahren, welche unsere weiteren Schritte ausführt."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "a6d15c94",
   "metadata": {},
   "outputs": [],
   "source": [
    "appName=\"jupyter-spark\"\n",
    "\n",
    "conf = SparkConf()\n",
    "\n",
    "# CLUSTER MANAGER\n",
    "################################################################################\n",
    "# set Kubernetes Master as Cluster Manager(“k8s://https://” is NOT a typo, this is how Spark knows the “provider” type).\n",
    "conf.setMaster(\"k8s://https://kubernetes.default.svc.cluster.local:443\")\n",
    "\n",
    "# CONFIGURE KUBERNETES\n",
    "################################################################################\n",
    "# set the namespace that will be used for running the driver and executor pods.\n",
    "conf.set(\"spark.kubernetes.namespace\",\"frontend\")\n",
    "# set the docker image from which the Worker pods are created\n",
    "conf.set(\"spark.kubernetes.container.image\", \"thinkportgmbh/workshops:spark-3.3.1\")\n",
    "conf.set(\"spark.kubernetes.container.image.pullPolicy\", \"Always\")\n",
    "\n",
    "# set service account to be used\n",
    "conf.set(\"spark.kubernetes.authenticate.driver.serviceAccountName\", \"spark\")\n",
    "# authentication for service account(required to create worker pods):\n",
    "conf.set(\"spark.kubernetes.authenticate.caCertFile\", \"/var/run/secrets/kubernetes.io/serviceaccount/ca.crt\")\n",
    "conf.set(\"spark.kubernetes.authenticate.oauthTokenFile\", \"/var/run/secrets/kubernetes.io/serviceaccount/token\")\n",
    "\n",
    "\n",
    "# CONFIGURE SPARK\n",
    "################################################################################\n",
    "conf.set(\"spark.sql.session.timeZone\", \"Europe/Berlin\")\n",
    "# set driver host. In this case the ingres service for the spark driver\n",
    "# find name of the driver service with 'kubectl get services' or in the helm chart configuration\n",
    "conf.set(\"spark.driver.host\", \"jupyter-spark-driver.frontend.svc.cluster.local\")\n",
    "# set the port, If this port is busy, spark-shell tries to bind to another port.\n",
    "conf.set(\"spark.driver.port\", \"29413\")\n",
    "# add the postgres driver jars into session\n",
    "conf.set(\"spark.jars\", \"/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar, /opt/spark/jars/spark-avro_2.12-3.3.1.jar\")\n",
    "#conf.set(\"spark.driver.extraClassPath\",\"/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar, /opt/spark/jars/spark-avro_2.12-3.3.1.jar\")\n",
    "conf.set(\"spark.executor.extraClassPath\",\"/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar, /opt/spark/jars/spark-avro_2.12-3.3.1.jar\")\n",
    "#conf.set(\"spark.executor.extraLibrary\",\"/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar\")\n",
    "\n",
    "# CONFIGURE S3 CONNECTOR\n",
    "conf.set(\"spark.hadoop.fs.s3a.endpoint\", \"minio.minio.svc.cluster.local:9000\")\n",
    "conf.set(\"spark.hadoop.fs.s3a.access.key\", \"trainadm\")\n",
    "conf.set(\"spark.hadoop.fs.s3a.secret.key\", \"train@thinkport\")\n",
    "conf.set(\"spark.hadoop.fs.s3a.path.style.access\", \"true\")\n",
    "conf.set(\"spark.hadoop.fs.s3.impl\", \"org.apache.hadoop.fs.s3a.S3AFileSystem\")\n",
    "conf.set(\"spark.hadoop.fs.s3a.aws.credentials.provider\", \"org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider\")\n",
    "conf.set(\"spark.hadoop.fs.s3a.connection.ssl.enabled\", \"false\")\n",
    "\n",
    "# conf.set(\"spark.jars.packages\", \"org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1\")\n",
    "\n",
    "# CONFIGURE WORKER (Customize based on workload)\n",
    "################################################################################\n",
    "# set number of worker pods\n",
    "conf.set(\"spark.executor.instances\", \"1\")\n",
    "# set memory of each worker pod\n",
    "conf.set(\"spark.executor.memory\", \"1G\")\n",
    "# set cpu of each worker pod\n",
    "conf.set(\"spark.executor.cores\", \"2\")\n",
    "# Number of possible tasks = cores * executores\n",
    "\n",
    "# SPARK SESSION\n",
    "################################################################################\n",
    "# and last, create the spark session and pass it the config object\n",
    "\n",
    "spark = SparkSession\\\n",
    "    .builder\\\n",
    "    .config(conf=conf) \\\n",
    "    .config('spark.sql.session.timeZone', 'Europe/Berlin') \\\n",
    "    .appName(appName)\\\n",
    "    .getOrCreate()\n",
    "\n",
    "# also get the spark context\n",
    "sc=spark.sparkContext\n",
    "# change the log level to warning, to see less output\n",
    "sc.setLogLevel('WARN')\n",
    "\n",
    "# get the configuration object to check all the configurations the session was startet with\n",
    "for entry in sc.getConf().getAll():\n",
    "        if entry[0] in [\"spark.app.name\",\"spark.kubernetes.namespace\",\"spark.executor.memory\",\"spark.executor.cores\",\"spark.driver.host\",\"spark.master\"]:\n",
    "            print(entry[0],\"=\",entry[1])"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "9f42ed16",
   "metadata": {},
   "source": [
    "## 3. Einlesen und Schreiben von Daten"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "93cbac52",
   "metadata": {},
   "source": [
    "### 3.1 Einlesen der Daten aus unserem S3 Speicher-Bucket \n",
    "Laden der Daten aus unserem Bucket in \"s3a://twitter/avro\" in einen DataFrame, um auf den Daten zu arbeiten. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "90b65543",
   "metadata": {},
   "outputs": [],
   "source": [
    "df=(spark\n",
    "    .read.format(\"avro\")\n",
    "    # Pfad zu Bucket\n",
    "    .load(\"s3a://twitter/avro\")\n",
    "    # filter retweets raus bei denen das Hashtag truncated wurde\n",
    "    .filter(f.array_contains(f.col(\"hashtags\"),\"BigData\")==True)\n",
    "    # repartition auf 20 um optimierter mit den wenigen cpu zu arbeiten\n",
    "    .repartition(20)\n",
    "   ).cache()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0f31f3ee",
   "metadata": {},
   "source": [
    "Erste Ausgabe der Daten in Form eines DataFrames"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "5dec3929",
   "metadata": {},
   "outputs": [],
   "source": [
    "df.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "7da71da4",
   "metadata": {},
   "source": [
    "### 3.2 Schreiben der Daten ins Delta-Format\n",
    "Hier werden die Daten direkt im Delta-Format umgewandelt und in den S3-Bucket \"s3a://twitter/delta\" geschrieben. Dieser Schritt ist wichtig, um die Daten passend für Trino zu abzulegen."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d6249e07",
   "metadata": {},
   "outputs": [],
   "source": [
    "writer_delta=(df\n",
    "                .write.partitionBy(\"language\")\n",
    "                .mode(\"overwrite\")\n",
    "                .format(\"delta\")\n",
    "                .option(\"overwriteSchema\", \"true\")\n",
    "                .option(\"userMetadata\", \"Initial Ladung\")\n",
    "                .save(\"s3a://twitter/delta\")\n",
    "             )"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "518be64b",
   "metadata": {},
   "source": [
    "## 4. Analyse-Aufgaben\n"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f9b79d70",
   "metadata": {},
   "source": [
    "### 4.1 Tweets anschauen und den Aufbau des Dataframes\n",
    "Schau dir den Datensatz einmal genau an. Welche Spalten gibt es? Welche Datentypen sind vorhanden?"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "406525a9",
   "metadata": {},
   "outputs": [],
   "source": [
    "df.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "9b4223d1",
   "metadata": {},
   "source": [
    "### 4.2  Das Schema des Datensatzes anzeigen \n",
    "<br>\n",
    "<code> df.printSchema()</code> gibt das Schema des Datensatzes aus."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "196c1696",
   "metadata": {},
   "outputs": [],
   "source": [
    "df.printSchema()"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "edc238b9",
   "metadata": {},
   "source": [
    "### 4.3 Zählen der Tweets pro Stunde\n",
    "Schreibe eine Abfrage, die **die Anzahl an Tweets pro Stunde** zählt.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "1185327b",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_hourly=(df  \n",
    "            ...\n",
    "          )\n",
    "\n",
    "df_hourly.show(20)"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "2f0d5a8f",
   "metadata": {},
   "source": [
    "### 4.4 Top 10 User nach Tweet-Anzahl\n",
    "Schreibe eine Abfrage, die die **Top User** nach ihrer **Anzahl an Tweets** ausgibt. Bedenke dabei, deine Ausgabe auf **10** Einträge zu limitieren."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "c2a6ca1f",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_top_user=(df\n",
    "                ...\n",
    "                )\n",
    "df_top_user.show()\n"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "75828392",
   "metadata": {},
   "source": [
    "### 4.5 Umgang mit  Arrays\n",
    "Für die folgenden Aufgabe wird die <code>explode</code>-Funktion benötigt. Schreibe eine Abfrage die das Hashtag-array mit <code>explode</code> teilt. Gebe dabei die Spalten \"user_name\", \"tweet_id\"und die explodierte\"hashtags\"- Spalte mit einem Limit von 20 Zeilen aus. \n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "bd44fd32",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_hash=(df\n",
    "         ...\n",
    "        )\n",
    "df_hash.show()"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "58765db1",
   "metadata": {},
   "source": [
    "### 4.6 Top 5 Hashtags der Top 10 User\n",
    "Schreibe eine Abfrage, die die **Top 5 der Hashtags** der **10 User** mit den **meisten Tweets** ausgibt."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f0bd4d64",
   "metadata": {
    "scrolled": true
   },
   "outputs": [],
   "source": [
    "df_top5_per_user=(df_top_user\n",
    "                ...\n",
    "                 )\n",
    "df_top5_per_user.show()"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "cbb68e6c",
   "metadata": {},
   "source": [
    " ### 4.7 Top 10 Influencer (User mit #BigData-tweets mit den meisten Followern) \n",
    " Schreibe eine Abfrage, die die **Top 10 Influencer** mit den **meisten Follower** zählt und sortiert anzeigt."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "79b5113b",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_top_influencer=(df\n",
    "                ...\n",
    "                )\n",
    "df_top_influencer.show(10)"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "935e6148",
   "metadata": {},
   "source": [
    "### 4.8 Top 10 Influencer und ihre Anzahl an tweets\n",
    "Schreibe eine Abfrage, die die **Top 10 Influencer**, ihre Follower und die **Anzahl ihrer Tweets** ausgibt. außeredem soll es sortiert nach den Anzahl ihrer Follower sein. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "e47a7c26",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_withRetweets=(df_top_user\n",
    "            ....\n",
    "    )\n",
    "\n",
    "df_withRetweets.show()"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "13676250",
   "metadata": {},
   "source": [
    "### Bonusaufgabe: Filter nach den Top 10 Locations und ihrem Top Hashtag\n",
    "Schreibe eine Abfrage, die die **Top 10 häufigsten Locations** ausgibt und das am **zweitmeisten verwendete Hashtag** dort. Da alle unsere Daten das Hashtag #BigData beinhalten. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "2f14e0c2",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "eb05cf17",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "132149f4",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "markdown",
   "id": "a081f82e",
   "metadata": {},
   "source": [
    "## 5. Delta History and Time Travel\n",
    "Führe den folgenden Code aus um die aktuelle Delta-Daten-Version upzudaten. Wenn du mehrere Versionen sehen willst schreibe öfter raus mit <code>writer_delta()</code> mit einigen Minuten Abstand."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "287ada1c",
   "metadata": {},
   "outputs": [],
   "source": [
    "writer_delta=(df\n",
    "                .write.partitionBy(\"language\")\n",
    "                .mode(\"overwrite\")\n",
    "                .format(\"delta\")\n",
    "                .option(\"overwriteSchema\", \"true\")\n",
    "                .option(\"userMetadata\", \"Update Ladung\")\n",
    "                .save(\"s3a://twitter/delta\")\n",
    "             )"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "52d10e78",
   "metadata": {},
   "source": [
    "### 5.1 Delta Tabelle ausgeben\n",
    "Lade die Delta-Tabelle und lasse dir die ersten 2 Einträge ausgeben."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "34f34013",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load Delta file in s3 into Delta Table Object\n",
    "dt = DeltaTable.forPath(spark, \"s3a://twitter/delta\")\n",
    "dt.toDF().show(2)\n"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "c7862d91",
   "metadata": {},
   "source": [
    "### 5.2  Erzeugen einer Historie\n",
    "1. Führe mehrmals Write to Delta aus und prüfe, wie die Historie neue Einträge hinzufügt  "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "5127f540",
   "metadata": {},
   "outputs": [],
   "source": [
    "# get the metadata for the full history of the table\n",
    "fullHistoryDF = dt.history()    \n",
    "\n",
    "# get the metadata for the last operation\n",
    "lastOperationDF = dt.history(1) \n",
    "\n",
    "fullHistoryDF.select(\"version\",\"readVersion\",\"timestamp\",\"userId\",\"operation\",\"operationParameters\",\"operationMetrics\").show()"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "39f0a855",
   "metadata": {},
   "source": [
    "### 5.3 Laden der aktuellen  Versionen \n",
    "Lade eine der Versionen und lasse dir alle `languages` anzeigen (via distinct().show())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f92161ce",
   "metadata": {},
   "outputs": [],
   "source": [
    "# load latest delta version\n"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "54cee9d7",
   "metadata": {},
   "source": [
    "### 5.4. Laden einer ältere Versionen \n",
    "Lade eine ältere Version und bestätige, dass noch alle Daten vorhanden sind."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "85e31aad",
   "metadata": {},
   "outputs": [],
   "source": [
    "#load specific historic version\n",
    "df_timetravel_old = spark.read.format(\"delta\").option(\"versionAsOf\", 2).load(\"s3a://twitter/delta\")\n",
    "df_timetravel_old.select(\"language\").distinct().show()\n",
    "df_timetravel_old.show()"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "9144a895",
   "metadata": {},
   "source": [
    "### 5.5 Überschreiben von neueren Version\n",
    "Überschreibe nun mit der älteren Version die Aktuellste. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "49e561fa",
   "metadata": {},
   "outputs": [],
   "source": [
    "# write old version back as latest\n"
   ]
  },
  {
   "attachments": {},
   "cell_type": "markdown",
   "id": "7cde3106",
   "metadata": {},
   "source": [
    "### 5.6 Zurück in die Zukunft\n",
    "Kehre zurück zum aktuellsten Timestamp, indem `timestampAsOf`anstelle von `versionAsOf`verwenden und einem aktuellen timestamp, anstelle der Versionsnummer."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8be4272d",
   "metadata": {
    "scrolled": true
   },
   "outputs": [],
   "source": [
    "f_b2future = (spark\n",
    "                .read.format(\"delta\").option(\"timestampAsOf\", \"2022-12-08 16:37:54\").load(\"s3a://twitter/delta\")\n",
    "               )\n",
    "f_b2future.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "5d5df865",
   "metadata": {},
   "source": [
    "# 6. Ausschalten der Spark-App\n",
    "**Bitte schließe am Ende die Spark-App wieder mit dem folgenden Befehl `spark.stop()`, wenn du fertig mit der Bearbeitung der Aufgaben bist.** "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "bfdb31e3",
   "metadata": {},
   "outputs": [],
   "source": [
    "spark.stop()\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "bb598b6c",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.2"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}